In [2]:
import numpy as np
import scipy.sparse as sp
# old = np.load
# np.load = lambda *a,**k: old(*a,**k,allow_pickle=True)
filename = 'pubmed.npz'
with np.load('../tensorflow1.11-graph2gauss/data/'+filename) as loader:
# with np.load('test_test.npz') as loader:
    loader = dict(loader)
    A = sp.csr_matrix((loader['adj_data'], loader['adj_indices'],
                       loader['adj_indptr']), shape=loader['adj_shape'])

    X = sp.csr_matrix((loader['attr_data'], loader['attr_indices'],
                       loader['attr_indptr']), shape=loader['attr_shape'])

    z = loader.get('labels')

    graph = {
        'A': A,
        'X': X,
        'z': z
    }

    idx_to_node = loader.get('idx_to_node')
    if idx_to_node:
        idx_to_node = idx_to_node.tolist()
        graph['idx_to_node'] = idx_to_node

    idx_to_attr = loader.get('idx_to_attr')
    if idx_to_attr:
        idx_to_attr = idx_to_attr.tolist()
        graph['idx_to_attr'] = idx_to_attr

    idx_to_class = loader.get('idx_to_class')
    if idx_to_class:
        idx_to_class = idx_to_class.tolist()
        graph['idx_to_class'] = idx_to_class

In [5]:
A, X, z = graph['A'], graph['X'], graph['z']

In [14]:
f1_micro, f1_macro, accuracy,precision_micro,precision_macro,recall_micro,recall_macro,average_precision_micro,average_precision_macro,roc_auc_micro,roc_auc_macro = score_node_classification(X, z, n_repeat=1, norm=True)
print('f1_micro: {:.4f}, f1_macro: {:.4f}'.format(f1_micro, f1_macro))
print('accuracy: {:.4f}'.format(accuracy))
print('precision_micro: {:.4f}, precision_macro: {:.4f}'.format(precision_micro, precision_macro))
print('recall_micro: {:.4f}, recall_macro: {:.4f}'.format(recall_micro, recall_macro))
print('average_precision_micro: {:.4f}, average_precision_macro: {:.4f}'.format(average_precision_micro, average_precision_macro))
print('roc_auc_micro: {:.4f}, roc_auc_macro: {:.4f}'.format(roc_auc_micro, roc_auc_macro))

/data/jtt_internship/.conda/envs/tensor2py8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/data/jtt_internship/.conda/envs/tensor2py8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

f1_micro: 0.8414, f1_macro: 0.8416
accuracy: 0.8414
precision_micro: 0.8414, precision_macro: 0.8428
recall_micro: 0.8414, recall_macro: 0.8405
average_precision_micro: 0.7613, average_precision_macro: 0.7609
roc_auc_micro: 0.8811, roc_auc_macro: 0.8785


In [13]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score,accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import label_binarize
def score_node_classification(features, z, p_labeled=0.1, n_repeat=10, norm=True):
    lrcv = LogisticRegressionCV()

    if norm:
        features = normalize(features)

    trace = []
    for seed in range(n_repeat):
        sss = StratifiedShuffleSplit(n_splits=1, test_size=1 - p_labeled, random_state=seed)
        split_train, split_test = next(sss.split(features, z))

        lrcv.fit(features[split_train], z[split_train])
        predicted = lrcv.predict(features[split_test])
        #one-hot multi-class
        labels = list(set(z[split_test]))
        y_binarize = label_binarize(z[split_test], classes=labels)
        predicted_binarize = label_binarize(predicted, classes=labels)

        f1_micro = f1_score(z[split_test], predicted, average='micro')
        f1_macro = f1_score(z[split_test], predicted, average='macro')
        precision_micro = precision_score(z[split_test], predicted, average='micro')
        precision_macro = precision_score(z[split_test], predicted, average='macro')
        accuracy= accuracy_score(z[split_test], predicted)
        recall_micro = recall_score(z[split_test], predicted, average='micro')
        recall_macro = recall_score(z[split_test], predicted, average='macro')

        average_precision_micro = average_precision_score(y_binarize, predicted_binarize, average='micro')
        average_precision_macro = average_precision_score(y_binarize, predicted_binarize, average='macro')
        roc_auc_micro = roc_auc_score(y_binarize, predicted_binarize, average='micro', multi_class='ovr')
        roc_auc_macro = roc_auc_score(y_binarize, predicted_binarize, average='macro', multi_class='ovr')
        
        trace.append((f1_micro, f1_macro, accuracy,precision_micro,precision_macro,recall_micro,recall_macro,average_precision_macro,\
            average_precision_micro,roc_auc_micro,roc_auc_macro))
    return np.array(trace).mean(0)